In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
CREATING SYLLABLE DICTIONARY...
CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [77]:
%run ../../../prepare_data.py
gen = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)

In [85]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])

(4, 11, 5168)
[0 0 0 ..., 0 0 0]
bolj
[0 0 0 ..., 0 0 0]
ši
[0 0 0 ..., 0 0 0]
mi
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]



In [84]:
# print(X_train[0])
test = next(gen)
# eye = np.eye(5168)[X_train[0]]
# print(X_train[0].tolist())
# np.eye(5168)[[2, 1]]



In [123]:
np.eye(10)[[3,7,2,5]]
print(eye[[2, 3]].shape)

(2, 5168)


In [11]:
X_validate.shape
# len(syllable_dictionary)

(54222, 11)

In [46]:
print(len(syllable_dictionary))
shuffle_vector_location = '../../internal_representations/inputs/shuffle_vector'
X_train, X_other_features_train, y_train = generate_syllable_X_and_y(syllable_dictionary, max_word, max_num_vowels, train_content, vowels, accetuated_vowels, feature_dictionary, shuffle_vector_location + '_train.h5')


5168
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
(430151, 11)
(430151,)
INPUTS SHUFFELED!


In [5]:
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

(430151, 23, 30)
(52058, 23, 30)
(54222, 23, 30)


NameError: name 'dictionary' is not defined

In [4]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 10
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [5]:
conv_input_shape=(11, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [6]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [93]:
# history = model.fit_generator(generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), X_test[:1000].shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), validation_steps=X_test[:1000].shape[0]/(batch_size * num_fake_epoch))

In [7]:
# history = model.fit_generator(generate_fake_epoch(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=([X_test, X_other_features_test], y_test))
history = model.fit_generator(generate_fake_epoch_syllables(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test, X_other_features_test, y_test, batch_size), validation_steps=X_test.shape[0]/(batch_size * num_fake_epoch))


# model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=1, batch_size=16)

Epoch 1/200
1345/1344 [==============================] - 359s - loss: 0.1989 - actual_accuracy: 0.2775 - val_loss: 0.1272 - val_actual_accuracy: 0.5721
Epoch 2/200
1345/1344 [==============================] - 182s - loss: 0.1079 - actual_accuracy: 0.6465 - val_loss: 0.0984 - val_actual_accuracy: 0.7147
Epoch 3/200
1345/1344 [==============================] - 188s - loss: 0.0836 - actual_accuracy: 0.7599 - val_loss: 0.0848 - val_actual_accuracy: 0.7918
Epoch 4/200
1345/1344 [==============================] - 194s - loss: 0.0705 - actual_accuracy: 0.8014 - val_loss: 0.0859 - val_actual_accuracy: 0.7814
Epoch 5/200
1345/1344 [==============================] - 202s - loss: 0.0606 - actual_accuracy: 0.8293 - val_loss: 0.0836 - val_actual_accuracy: 0.7975
Epoch 6/200
1345/1344 [==============================] - 198s - loss: 0.0527 - actual_accuracy: 0.8553 - val_loss: 0.0816 - val_actual_accuracy: 0.8025
Epoch 7/200
1345/1344 [==============================] - 204s - loss: 0.0471 - actual_ac

1345/1344 [==============================] - 228s - loss: 0.0075 - actual_accuracy: 0.9829 - val_loss: 0.1420 - val_actual_accuracy: 0.8650
Epoch 55/200
1345/1344 [==============================] - 226s - loss: 0.0070 - actual_accuracy: 0.9848 - val_loss: 0.1336 - val_actual_accuracy: 0.8700
Epoch 56/200
1345/1344 [==============================] - 222s - loss: 0.0067 - actual_accuracy: 0.9846 - val_loss: 0.1662 - val_actual_accuracy: 0.8566
Epoch 57/200
1345/1344 [==============================] - 230s - loss: 0.0068 - actual_accuracy: 0.9852 - val_loss: 0.1388 - val_actual_accuracy: 0.8578
Epoch 58/200
1345/1344 [==============================] - 227s - loss: 0.0063 - actual_accuracy: 0.9865 - val_loss: 0.1476 - val_actual_accuracy: 0.8524
Epoch 59/200
1345/1344 [==============================] - 227s - loss: 0.0071 - actual_accuracy: 0.9845 - val_loss: 0.1395 - val_actual_accuracy: 0.8700
Epoch 60/200
1345/1344 [==============================] - 230s - loss: 0.0070 - actual_accuracy

1345/1344 [==============================] - 247s - loss: 0.0034 - actual_accuracy: 0.9927 - val_loss: 0.1871 - val_actual_accuracy: 0.8620
Epoch 108/200
1345/1344 [==============================] - 246s - loss: 0.0033 - actual_accuracy: 0.9926 - val_loss: 0.1899 - val_actual_accuracy: 0.8593
Epoch 109/200
1345/1344 [==============================] - 247s - loss: 0.0029 - actual_accuracy: 0.9935 - val_loss: 0.2048 - val_actual_accuracy: 0.8574
Epoch 110/200
1345/1344 [==============================] - 252s - loss: 0.0039 - actual_accuracy: 0.9910 - val_loss: 0.1691 - val_actual_accuracy: 0.8704
Epoch 111/200
1345/1344 [==============================] - 249s - loss: 0.0034 - actual_accuracy: 0.9921 - val_loss: 0.1848 - val_actual_accuracy: 0.8608
Epoch 112/200
1345/1344 [==============================] - 247s - loss: 0.0030 - actual_accuracy: 0.9938 - val_loss: 0.2009 - val_actual_accuracy: 0.8535
Epoch 113/200
1345/1344 [==============================] - 245s - loss: 0.0027 - actual_ac

1345/1344 [==============================] - 259s - loss: 0.0022 - actual_accuracy: 0.9952 - val_loss: 0.2302 - val_actual_accuracy: 0.8535 - los - ETA: 2s - loss: 0.0022 - actual_
Epoch 161/200
1345/1344 [==============================] - 266s - loss: 0.0020 - actual_accuracy: 0.9955 - val_loss: 0.2546 - val_actual_accuracy: 0.8344
Epoch 162/200
1345/1344 [==============================] - 260s - loss: 0.0022 - actual_accuracy: 0.9952 - val_loss: 0.2386 - val_actual_accuracy: 0.8462
Epoch 163/200
1345/1344 [==============================] - 259s - loss: 0.0019 - actual_accuracy: 0.9960 - val_loss: 0.2581 - val_actual_accuracy: 0.8466
Epoch 164/200
1345/1344 [==============================] - 254s - loss: 0.0019 - actual_accuracy: 0.9959 - val_loss: 0.2220 - val_actual_accuracy: 0.8581
Epoch 165/200
1345/1344 [==============================] - 257s - loss: 0.0018 - actual_accuracy: 0.9964 - val_loss: 0.2294 - val_actual_accuracy: 0.8497
Epoch 166/200
1345/1344 [========================

In [8]:
name = '10_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [46]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
feature_dictionary = create_feature_dictionary()
syllable_dictionary = create_syllables_dictionary(content, vowels)
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [5]:
# create_syllables(content[375488][0], vowels)
create_syllables('izzvati', vowels)

['iz', 'zva', 'ti']

In [45]:

syllables_dictionary = create_syllables_dictionary(content, vowels)

NameError: name 'content' is not defined

In [9]:
syllables_dictionary = sorted(syllables_dictionary)
syllables_dictionary

['',
 '-o',
 'a',
 'ab',
 'ah',
 'akt',
 'al',
 'alf',
 'alg',
 'alt',
 'am',
 'an',
 'ant',
 'ar',
 'arh',
 'as',
 'at',
 'av',
 'ba',
 'bab',
 'bah',
 'baj',
 'bajt',
 'bak',
 'bal',
 'bam',
 'ban',
 'banj',
 'bank',
 'bar',
 'bark',
 'barv',
 'bas',
 'bast',
 'bat',
 'bav',
 'baz',
 'bač',
 'baš',
 'baž',
 'bde',
 'bdel',
 'bdet',
 'bdeč',
 'bdi',
 'bdim',
 'bdiš',
 'be',
 'bec',
 'bed',
 'beg',
 'beh',
 'bej',
 'bek',
 'bel',
 'belc',
 'belj',
 'belk',
 'bem',
 'ben',
 'benj',
 'benk',
 'bent',
 'ber',
 'bes',
 'bet',
 'bev',
 'bez',
 'beč',
 'beš',
 'bež',
 'bi',
 'bic',
 'bih',
 'bij',
 'bik',
 'bil',
 'bilj',
 'bilk',
 'bim',
 'bin',
 'bir',
 'bisk',
 'bit',
 'bitk',
 'biv',
 'bič',
 'biš',
 'bišč',
 'bja',
 'bjav',
 'bje',
 'bjej',
 'bjekt',
 'bjel',
 'bjem',
 'bjest',
 'bjet',
 'bješ',
 'bji',
 'bjih',
 'bjim',
 'bjo',
 'bju',
 'bla',
 'blag',
 'blah',
 'blak',
 'blam',
 'blan',
 'blast',
 'blat',
 'ble',
 'bled',
 'blej',
 'blek',
 'blem',
 'blesk',
 'blest',
 'blet',
 'bleč'

In [57]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)